In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/orgspec-weak_supervision/')  

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
for dirname, _, filenames in os.walk('./demos/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./demos/input/training-Ovolvulus.csv
./demos/input/02_holdout.csv
./demos/input/ov_training.csv
./demos/input/ov_holdout.csv


In [ ]:
import pandas as pd 

# Importing the dataset
#dataset_ovolvulus = pd.read_csv('demos/input/training-Ovolvulus.csv')

#dataset_ovolvulus["differ"] = abs(dataset_ovolvulus["Info_n_Positive"] - dataset_ovolvulus["Info_n_Negative"]) 

#val_ovolvulus = dataset_ovolvulus.loc[dataset_ovolvulus["differ"]>= 1]

#train_ovolvulus = dataset_ovolvulus.loc[dataset_ovolvulus["differ"] == 0]

#train_ovolvulus = dataset_ovolvulus


In [ ]:
feature_columns = {'feat_VHSE5', 'feat_BLOSUM1', 'feat_F6', 'feat_MSWHIM2', 'feat_PP2', 'feat_KF3', 'feat_VHSE1', 'feat_Z1', 'feat_Z4', 'feat_F1', 'feat_BLOSUM7', 'feat_ST5', 'feat_O_atoms', 'feat_ProtFP4', 'feat_PP3', 'feat_PP1', 'feat_ST7'}
target = 'Class'

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.utils import shuffle

# Importing the dataset
train_ovolvulus = pd.read_csv('./demos/input/ov_training.csv')

train_ovolvulus.replace([np.inf, -np.inf], np.nan, inplace=True)
train_ovolvulus.dropna(inplace=True)

train_ovolvulus = shuffle(train_ovolvulus, random_state=1)

In [ ]:
train_ovolvulus

,Info_epitope_id,Info_sourceOrg_id,Info_protein_id,Info_host_id,Info_n_Positive,Info_n_Negative,Info_TSeq_accver,Info_TSeq_taxid,Info_TSeq_orgname,Info_center_pos,Info_window_seq,Info_evidence_type,Class,feat_seq_entropy,feat_C_atoms,feat_H_atoms,feat_N_atoms,feat_O_atoms,feat_S_atoms,feat_molecular_weight,feat_Perc_Tiny,feat_Perc_Small,feat_Perc_Aliphatic,feat_Perc_Aromatic,feat_Perc_NonPolar,feat_Perc_Polar,feat_Perc_Charged,feat_Perc_Basic,feat_Perc_Acidic,feat_PP1,feat_PP2,feat_PP3,feat_KF1,feat_KF2,feat_KF3,feat_KF4,feat_KF5,feat_KF6,feat_KF7,feat_KF8,...,feat_Perc_WA,feat_Perc_WC,feat_Perc_WD,feat_Perc_WE,feat_Perc_WF,feat_Perc_WG,feat_Perc_WH,feat_Perc_WI,feat_Perc_WK,feat_Perc_WL,feat_Perc_WM,feat_Perc_WN,feat_Perc_WP,feat_Perc_WQ,feat_Perc_WR,feat_Perc_WS,feat_Perc_WT,feat_Perc_WV,feat_Perc_WW,feat_Perc_WY,feat_Perc_YA,feat_Perc_YC,feat_Perc_YD,feat_Perc_YE,feat_Perc_YF,feat_Perc_YG,feat_Perc_YH,feat_Perc_YI,feat_Perc_YK,feat_Perc_YL,feat_Perc_YM,feat_Perc_YN,feat_Perc_YP,feat_Perc_YQ,feat_Perc_YR,feat_Perc_YS,feat_Perc_YT,feat_Perc_YV,feat_Perc_YW,feat_Perc_YY
56317,853549,6282,AAC47233.1,"9606,9606",2,0,AAC47233.1,6282,Onchocerca volvulus,97,FEDENFKEKHTGPGV,Epitope-containing region,1,3.189898,77,140,20,40,0,1986.07,0.200000,0.466667,0.066667,0.200000,0.400000,0.600000,0.466667,0.200000,0.266667,0.109333,-0.366667,-0.106667,0.024000,-0.084000,-0.423333,0.340667,0.129333,-0.226667,0.160667,0.445333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
58152,856014,6282,AAC48290.1,"9606,9606",0,2,AAC48290.1,6282,Onchocerca volvulus,346,YVESANMQSPMEMIP,Epitope-containing region,-1,3.189898,73,143,17,39,3,1979.20,0.200000,0.466667,0.200000,0.066667,0.600000,0.400000,0.133333,0.000000,0.133333,-0.158000,-0.341333,-0.119333,-0.156000,-0.186000,-0.190000,-0.040000,0.335333,0.154667,-0.265333,-0.251333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0
51462,856242,6282,AAA65186.1,"9606,9606",1,1,AAA65186.1,6282,Onchocerca volvulus,74,LKPDAKAFVDEIIAK,Epitope-containing region,1,3.006239,77,156,18,36,0,1910.15,0.200000,0.466667,0.466667,0.066667,0.600000,0.400000,0.400000,0.200000,0.200000,-0.238667,-0.314667,-0.030000,-0.492000,-0.212000,-0.300000,0.124000,-0.090000,-0.704000,-0.063333,-0.002667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
50515,851100,6282,AAA50214.2,"9606,9606",1,1,AAA50214.2,6282,Onchocerca volvulus,144,PTIDDNEAKKLFGEK,Epitope-containing region,1,3.323231,75,149,19,40,0,1957.09,0.200000,0.466667,0.200000,0.066667,0.400000,0.600000,0.466667,0.200000,0.266667,0.104000,-0.364667,-0.057333,-0.092000,-0.100667,-0.476000,0.401333,-0.038667,-0.490000,0.190667,0.158000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
55240,852724,6282,AAB69626.2,"9606,9606",2,0,AAB69626.2,6282,Onchocerca volvulus,49,PTFNAGGSFAGAILG,Epitope-containing region,1,2.923231,63,122,16,33,0,1631.73,0.600000,0.733333,0.333333,0.133333,0.800000,0.200000,0.000000,0.000000,0.000000,-0.608000,-0.518000,0.146667,0.216000,-0.882000,-0.193333,-0.338000,-0.152667,-0.448000,0.314000,0.518667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
#train_ovolvulus = train_ovolvulus.iloc[0:int(len(train_ovolvulus))]

noisy_train_ovolvulus = train_ovolvulus.loc[train_ovolvulus['Info_evidence_type']=='Epitope-containing region']
exact_train_ovolvulus = train_ovolvulus.loc[train_ovolvulus['Info_evidence_type']=='Exact Epitope']

noisy_train_ovolvulus = shuffle(noisy_train_ovolvulus, random_state=1)
exact_train_ovolvulus = shuffle(exact_train_ovolvulus, random_state=1)

# get noisy and non-noisy 
df_X_1_noisy = noisy_train_ovolvulus.iloc[0:25]
df_X_1_exact = exact_train_ovolvulus.iloc[0:25]

# small dataset
df_X_1 = pd.concat([df_X_1_noisy, df_X_1_exact])
df_X_1 = shuffle(df_X_1, random_state=1)


#df_X_2_part1 = noisy_train_ovolvulus.iloc[51:16963]
#df_X_2_part2 = exact_train_ovolvulus[51:1874]

# unlabeld dataset
#df_X_2 = pd.concat([df_X_2_part1, df_X_2_part2])
#df_X_2 = shuffle(df_X_2, random_state=1)

df_X_2 = train_ovolvulus

X_1 = df_X_1[feature_columns]
y_1 = df_X_1[target]

X_2 = df_X_2[feature_columns]
y_2 = df_X_2[target]


y_1_2 = np.concatenate((y_1, y_2.apply(lambda x: -1)))
X_1_2 = np.concatenate((X_1, X_2))

In [ ]:
# Importing the dataset
holdout_ovolvulus = pd.read_csv('./demos/input/ov_holdout.csv')

holdout_ovolvulus.replace([np.inf, -np.inf], np.nan, inplace=True)
holdout_ovolvulus.dropna(inplace=True)

holdout_ovolvulus = shuffle(holdout_ovolvulus, random_state=1)

holdout_ovolvulus = holdout_ovolvulus.iloc[0:int(len(holdout_ovolvulus)/2)]

df_X_3 = holdout_ovolvulus

#df_X_3['Class'] = df_X_3['Class'].apply(lambda x: 0 if x == -1 else x)

X_3 = df_X_3[feature_columns]
y_3 = df_X_3[target]

In [ ]:
df_X_2.Class.value_counts()

 1    10289
-1     8548
Name: Class, dtype: int64

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import warnings
warnings.filterwarnings("ignore")

try:
    import Asterisk.heuristics_generator
except ImportError:
    print(ImportError)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Reading and preparing the data from csv file

In [ ]:
!pip install sqlalchemy==1.2.2
!pip install asterisk
!pip install numbskull

In [ ]:
from Asterisk.heuristics_generator.hg_utils import *
from Asterisk.heuristics_generator.extra import *

feature_columns_ws = {'feat_VHSE5', 'feat_BLOSUM1', 'feat_F6', 'feat_MSWHIM2', 'feat_PP2', 'feat_KF3', 'feat_VHSE1', 'feat_Z1', 'feat_Z4', 'feat_F1', 'feat_BLOSUM7', 'feat_ST5', 'feat_O_atoms', 'feat_ProtFP4', 'feat_PP3', 'feat_PP1', 'feat_ST7', 'Class'}

In [ ]:
df_features, df_ground = split_features(df_X_2[feature_columns_ws], 'Class')
train_set, train_ground = split_sets(df_features, df_ground)


Clearing existing...
Running UDF...
Set sent : # 18837


In [ ]:
df_features, df_ground = split_features(df_X_1[feature_columns_ws], 'Class')
dev_set, val_ground = split_sets(df_features, df_ground)

Clearing existing...
Running UDF...
Set sent : # 50


In [ ]:
df_features, df_ground = split_features(df_X_3[feature_columns_ws], 'Class')
test_set, test_ground = split_sets(df_features, df_ground)

Clearing existing...
Running UDF...
Set sent : # 2664


## Heuristics Generator
The system starts with the heuristics generator component which takes the labeled set DL and the unlabeled set DU as inputs and outputs a set of heuristics H of size K denoted as (h1, h2, …hk) and a vector of initial probabilistic labels for the points in DU.
The process starts with defining the input (features) for the potential models. Then, the process continues with creating the models (heuristics) and evaluating their performance and coverage. Finally, the process ranks the heuristics generated by each, and every, iteration to decide upon which heuristic to add to the set H.

When evaluating the performance of the heuristics produced during each iteration, the component also considers the overall coverage of the heuristics when applied to DU. The component aims to widen the range of the data points that receive labels from H in DU. In other words, the goal of the component is to output a set of heuristics that are individually accurate while achieving high labeling coverage when combined. Therefore, to estimate the performance of the heuristics. The performance metrics are computed by applying each heuristic to DL.

In [ ]:
from Asterisk.heuristics_generator.loader import DataLoader
from Asterisk.heuristics_generator.verifier import Verifier
from Asterisk.heuristics_generator.heuristic_generator import HeuristicGenerator
from Asterisk.heuristics_generator.synthesizer import Synthesizer

#df_gold_label = prepare_hg(DU)
dl = DataLoader()
train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, \
train_ground, val_ground, test_ground = dl.load_data_tabular(train_set, test_set, dev_set, train_ground, test_ground, val_ground)

In [ ]:

validation_accuracy = []
training_accuracy = []
validation_coverage = []
training_coverage = []
training_marginals = []
idx = None

hg = HeuristicGenerator(train_primitive_matrix, val_primitive_matrix, val_ground, train_ground, b=0.5)
for i in range(3,52):
    if i == 3:
        hg.run_synthesizer(max_cardinality=3, idx=idx, keep=3, model='dt')
    else:
        hg.run_synthesizer(max_cardinality=3, idx=idx, keep=1, model='dt')
    hg.run_verifier()
   
    va,ta, vc, tc = hg.evaluate()
    validation_accuracy.append(va)
    training_accuracy.append(ta)
    training_marginals.append(hg.vf.train_marginals)
    validation_coverage.append(vc)
    training_coverage.append(tc)
        
    hg.find_feedback()
    idx = hg.feedback_idx
    
    if idx == []:
        break

In [ ]:
training_marginals = training_marginals
L_train = hg.L_train

from scipy import sparse
L_train = L_train.astype(int)
L_train= sparse.csr_matrix(L_train) 

Finally, to combine the output of the heuristic and generate an initial vector of probabilistic labels, we employ a generative model to learn the accuracies of the heuristics in H and produce a single probabilistic label for each data point in the unlabeled dataset. 

In [ ]:
global gen_model 
gen_model, gen_train_marginals = Fitting_Gen_Model(L_train)

Inferred cardinality: 2
[1.97992573 1.97838267 1.97760138]
0.0


In [ ]:
predictions = gen_model.predictions(L_train, batch_size=None)
#gen_model.learned_lf_stats()

In [ ]:
# Training the model with labeled trainig dataset
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier();

model.fit(X_2, predictions)

preds = model.predict(X_3)

print("ROC_AUC:", roc_auc_score(y_3, preds))

print(classification_report(y_3, preds))

ROC_AUC: 0.6686952545618221
              precision    recall  f1-score   support

          -1       0.66      0.56      0.61      1175
           1       0.69      0.78      0.73      1489

    accuracy                           0.68      2664
   macro avg       0.68      0.67      0.67      2664
weighted avg       0.68      0.68      0.68      2664

